In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database.downloader.downloader_base import DownloaderBase
import database.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [2]:
def plot_series_dist(series):
    data = series
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_label(self, stock_dataframe):
        N = 5 # 最大持仓周期 = N天，第N+1天开盘卖出
        df = stock_dataframe.copy()
        # 标签构建
        df['future_return'] = df['close'].shift(-N) / df['open'].shift(-1) - 1 # 计算第N日收益率
        # 极值处理
        df['future_return'] = np.clip(
            df['future_return'], 
            np.nanquantile(df['future_return'], 0.01), 
            np.nanquantile(df['future_return'], 0.99),
            )
        # 过滤第二天一字涨停情况
        df = df[df['high'].shift(-1) != df['low'].shift(-1)]
        return df[['datetime', 'future_return']]


    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_base = self.db_downloader._download_stock_base_info(stock_code) # 获取基础代码
        stock_individual = self.db_downloader._download_stock_individual_info(stock_code) # 获取profile信息
        stock_history = self.db_downloader._download_stock_history_info(stock_code, start_date, end_date) # 获取历史行情
        stock_indicator = self.db_downloader._download_stock_indicator_info(stock_code, start_date, end_date) # 获取指标数据
        stock_factor_date = self.db_downloader._download_stock_factor_date_info() # 获取日期特征
        stock_factor_qlib = self.db_downloader._download_stock_factor_qlib_info(stock_code, start_date, end_date) # 获取量价特征
        stock_label = self._build_label(stock_history) # 构建label
        stock_df = stock_base.merge(stock_individual, on=['stock_code']).merge(stock_history, on=['stock_code']).merge(stock_indicator, on=['stock_code', 'datetime']).merge(stock_label, on=['datetime']).merge(stock_factor_date, on=['datetime']).merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_base \
            .merge(stock_individual, on=['stock_code', 'stock_name']) \
            .merge(stock_history, on=['stock_code']) \
            .merge(stock_indicator, on=['stock_code', 'datetime']) \
            .merge(stock_label, on=['datetime']) \
            .merge(stock_factor_date, on=['datetime']) \
            .merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_df.dropna()
        return stock_df
    
    def _process_all_stock(self, code_type, start_date, end_date):
        # stock_code_list = list(ak.stock_info_a_code_name()['code'].unique()) # 获取A股所有股票列表
        stock_code_list = list(ak.index_stock_cons(code_type)['品种代码'].unique()) # 获取沪深300的股票代码列表
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {code_type} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)
        # return stock_df_list

In [3]:
db_conn = sqlite3.connect('../database/hh_quant.db')
db_downloader = DownloaderBase(db_conn, db_config)
proprocessor = PreProcessing(db_downloader=db_downloader)

## 使用Tensorflow

In [4]:
# 使用tensorflow处理原始数据
import numpy as np
import pandas as pd
import tensorflow as tf
from model import QuantModel
print(tf.__version__)

2024-03-20 18:42:59.213737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.15.0


In [5]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    return train_data, val_data

def transfer_data_type(df, columns, dtype):
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

def get_numeric_boundaries(series, num_bins=20):
    if series.nunique() < num_bins:
        boundaries = sorted(series.unique())
    else:
        boundaries = sorted(pd.qcut(series, num_bins, retbins=True, duplicates='drop')[1].tolist())
    return boundaries

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=True, batch_size=32):
    features = dataframe[feature_cols]
    labels = [dataframe[label_col] for label_col in label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), tuple(labels)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(features))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [6]:
backtest_start_date = '20190101'
backtest_end_date = '20240101'
train_period = 4 # year：训练数据周期长度
update_period = 6 # month：模型更新周期长度

def get_rolling_date_period(backtest_start_date, backtest_end_date, training_period, update_period):
    backtest_start_date = datetime.strptime(backtest_start_date, '%Y%m%d')
    backtest_end_date = datetime.strptime(backtest_end_date, '%Y%m%d')
    result = []
    rolling_flag = True
    while rolling_flag:
        current_val_start_date = backtest_start_date
        current_val_end_date = current_val_start_date + relativedelta(months=update_period) - relativedelta(days=1)
        if current_val_start_date < backtest_end_date:
            current_train_start_date = current_val_start_date - relativedelta(years=training_period)
            current_train_end_date = current_val_start_date - relativedelta(days=1)
            result.append([
                current_train_start_date.strftime("%Y%m%d"),
                current_train_end_date.strftime("%Y%m%d"),
                current_val_start_date.strftime("%Y%m%d"),
                current_val_end_date.strftime("%Y%m%d")
                ])
            backtest_start_date += relativedelta(months=update_period) 
        else:
            rolling_flag=False # 结束滚动训练
    return result

rolling_period = get_rolling_date_period(backtest_start_date, backtest_end_date, train_period, update_period)
rolling_period

[['20150101', '20181231', '20190101', '20190630'],
 ['20150701', '20190630', '20190701', '20191231'],
 ['20160101', '20191231', '20200101', '20200630'],
 ['20160701', '20200630', '20200701', '20201231'],
 ['20170101', '20201231', '20210101', '20210630'],
 ['20170701', '20210630', '20210701', '20211231'],
 ['20180101', '20211231', '20220101', '20220630'],
 ['20180701', '20220630', '20220701', '20221231'],
 ['20190101', '20221231', '20230101', '20230630'],
 ['20190701', '20230630', '20230701', '20231231']]

In [7]:
# df = proprocessor._process_all_stock(code_type='000016', start_date='20170701', end_date='20231231')

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeaturePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.means_ = None
        self.stds_ = None

    def fit(self, X, y=None):
        self.means_ = X.mean()
        self.stds_ = X.std()
        return self

    def transform(self, X, y=None):
        # 检查是否已经拟合
        if self.means_ is None or self.stds_ is None:
            raise RuntimeError("The model has not been fitted yet!")

        X_clipped = X.clip(self.means_ - 3 * self.stds_, self.means_ + 3 * self.stds_, axis=1) # 极值处理
        X_standardized = (X_clipped - self.means_) / self.stds_ # 标准化
        return X_standardized

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)
    
    def inverse_transform(self, X_standardized, y=None):
        # 检查是否已经拟合
        if self.means_ is None or self.stds_ is None:
            raise RuntimeError("The model has not been fitted yet!")
        
        # 反标准化
        X_original = (X_standardized * self.stds_) + self.means_
        return X_original

In [9]:
feature_config = {
    "target_feature_name": ["future_return"],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME0', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['weekday', 'day_of_month', 'month'],
    "string_categorical_features": ['industry', 'day_of_week', 'season']
}
full_feature_names = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
benchmark = '000016' # [上证50:000016, 沪深300:000300, 中证500:000905]所有股票作为训练数据
batch_size = 256

for date_period in tqdm(rolling_period[:1], desc='Rolling Training...'):
    train_start_date, train_end_date, val_start_date, val_end_date = date_period
    print(f"train_start: {train_start_date}, train_end: {train_end_date}, val_start: {val_start_date}, val_end: {val_end_date}")
    # 1. 获取所有股票信息
    df = proprocessor._process_all_stock(code_type=benchmark, start_date=train_start_date, end_date=val_end_date)
    # 2. 拆分训练数据&验证数据
    train_data, val_data = extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date)
    # 2.1 特征工程
    feature_fp = FeaturePreprocessor()
    norm_feature_columns = feature_config.get('numeric_features', [])
    train_data[norm_feature_columns] = feature_fp.fit_transform(train_data[norm_feature_columns])
    val_data[norm_feature_columns] = feature_fp.transform(val_data[norm_feature_columns])
    
    # 3. 构建训练集和验证集
    train_ds = df_to_dataset(train_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=True, batch_size=batch_size)
    val_ds = df_to_dataset(val_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=False, batch_size=batch_size)
    # 4. 配置模型相关参数
    model_config = {
        "seed": 1024,
        "reduction_ratio": 3,
        "cin_size": [32, 32],
        "dnn_hidden_units": [128,64],
        "dnn_activation": 'relu',
        "dnn_dropout": 0.2,
        "dnn_use_bn": True,
        "numeric_features_with_boundaries": {k: list(get_numeric_boundaries(train_data[k])) for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('string_categorical_features', [])},
        "feature_embedding_dims": 4,
        "task_type": ['reg'],
    }
    # 5. 初始化模型
    model = QuantModel(model_config)

    # 8. 配置optimizer
    initial_learning_rate = 1e-3
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate,
        decay_steps=(len(train_data) // batch_size)*10,
        decay_rate=1,
        staircase=False)
    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(lr_schedule), 
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.MeanSquaredError()
        ]
    )
    model.fit(
            train_ds, 
            validation_data=val_ds, 
            epochs=50,
            verbose=2,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                tf.keras.callbacks.TensorBoard(log_dir="./logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=1)
            ],
    )
    # 9. 配置保存模型功能
    # model_save_path = f'./models/saved_model/model_of_{val_start_date}'
    # model.save(model_save_path)
    # best_model = tf.keras.models.load_model('./best_model')

    # 10. 记录预测集合
    model_pred_dir = f'../../Offline/backtest/backtest_data/{benchmark}/'
    os.makedirs(model_pred_dir, exist_ok=True)
    model_red_result = model.predict(val_ds)
    output_df = val_data[['stock_code', 'stock_name', 'datetime']]
    output_df['future_return'] = val_data['future_return']
    output_df['future_return_pred'] = model_red_result
    output_file_path = f'{model_pred_dir}/stock_selection_results_{val_start_date}.pkl'
    output_df.to_pickle(output_file_path)

Rolling Training...:   0%|          | 0/1 [00:00<?, ?it/s]

train_start: 20150101, train_end: 20181231, val_start: 20190101, val_end: 20190630


Process: 000016 ...: 100%|██████████| 50/50 [00:05<00:00,  9.10it/s]


train_data_size: (32394, 207)
validation_data_size: (4185, 207)
Epoch 1/50
dnn_output: (None, 64)
cin_output: (None, 254)
dnn_output: (None, 64)
cin_output: (None, 254)
dnn_output: (None, 64)
cin_output: (None, 254)
127/127 - 99s - loss: 0.8036 - mean_absolute_error: 0.6646 - mean_squared_error: 0.7502 - val_loss: 0.0681 - val_mean_absolute_error: 0.1197 - val_mean_squared_error: 0.0212 - 99s/epoch - 783ms/step
Epoch 2/50
127/127 - 80s - loss: 0.2672 - mean_absolute_error: 0.3701 - mean_squared_error: 0.2236 - val_loss: 0.0534 - val_mean_absolute_error: 0.0927 - val_mean_squared_error: 0.0133 - 80s/epoch - 631ms/step
Epoch 3/50
127/127 - 80s - loss: 0.1392 - mean_absolute_error: 0.2475 - mean_squared_error: 0.1010 - val_loss: 0.0452 - val_mean_absolute_error: 0.0740 - val_mean_squared_error: 0.0088 - 80s/epoch - 631ms/step
Epoch 4/50
127/127 - 81s - loss: 0.0879 - mean_absolute_error: 0.1774 - mean_squared_error: 0.0526 - val_loss: 0.0418 - val_mean_absolute_error: 0.0683 - val_mean_sq

In [ ]:
# model.summary()

In [ ]:
# model_config